In [6]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wquantiles

from pathlib import Path
from scipy.stats import trim_mean
from statsmodels import robust

%matplotlib inline

#禁用科学计数法
np.set_printoptions(suppress=True,   precision=10,  threshold=2000,  linewidth=150)  
pd.set_option('display.float_format',lambda x : '%.2f' % x)

#设置数据文件夹路径
DATA = Path().resolve()  / 'data'

# 引入

我已经数不清自己是多少次从头开始学统计学了。

虽然每次都没有毅力坚持学完一本书的内容，但是我却在一次又一次的学习过程中，收纳了很多相关的资料。

这是我最近一次（或许也是最后一次）系统性地学习统计学了吧。

但是这次我打算，换一种方法从头开始学统计学。

如果你觉得你也有学习统计学的困惑的话，欢迎你阅读这份文档。

## 准备工作

如果你能看到这份文档，我猜你应该知道 Conda 和 Jupyter Notebook 的基础操作了吧？

这方面的内容我就不赘述了。

但是，我希望你能在继续阅读这份文档之前，提前做好一些准备工作：

### 阅读这两篇文章吧～

- [说说统计学、概率论和数理统计这些老梗](https://blog.sciencenet.cn/blog-242272-1036214.html)
- [如何学习统计学，或我的学习之路——初学者写给初学者](https://cosx.org/2008/11/how-to-learn-statistics-by-jthu/)

### 参考书目

- 《你一定爱读的极简统计学》（频率学派）[豆瓣](https://book.douban.com/subject/26323362/)、

- 《统计学关我什么事》（贝叶斯学派）[豆瓣](https://book.douban.com/subject/30229508/)、

- 《白话统计学》[豆瓣](https://book.douban.com/subject/25896013/)、[英文官网](https://www.routledge.com/Statistics-in-Plain-English/Urdan/p/book/9780367342838)、[]()

- 《数据科学中的实用统计学（第2版）》[豆瓣](https://book.douban.com/subject/35634945/)、[英文官网](https://www.oreilly.com/library/view/practical-statistics-for/9781492072935/)、[Github](https://github.com/gedeck/practical-statistics-for-data-scientists)、[]()

- 《利用Python进行数据分析（原书第2版）》[豆瓣](https://book.douban.com/subject/30283996/)、

- 《商务与经济统计（原书第13版）》[豆瓣](https://book.douban.com/subject/27126044/)、

如果你没有获取渠道，也没有关系，因为本文档会摘录参考书中的重要知识点。如果你需要深入了解，就需要阅读参考书目。

### 玩一玩这几个网站
- [Seeing theory](https://charts.upgrad.com/)

我们先不需要考虑“描述统计”这个概念具体是什么含义，我们先来看一下案例数据集长啥样：

导入案例所需要使用的库，以及设置数据集访问路径：

注：案例数据集存储路径是文档所在文件夹中的 'data' 子文件夹

# 基础概念

- 数据（data）是为了描述和解释所搜集、分析、汇总的事实和数字。
- 将用于特定研究而搜集的所有数据称为研究的数据集（dataset）。
- 个体（element）是指搜集数据的实体。
- 变量（variable）是个体中所感兴趣的那些特征。
- 对每个个体的每一变量收集测量值，从而得到数据。对某一特定个体得到的测量值集合称为一个观测值（observation）。


- 总体指的是数据集中的每一行数据，由于数据集是美国 50 个州的犯罪数据，所以数据集一共有50行
- 样本指的是对总体进行抽样之后，获得的子集，获取样本的方式有三种
    - 随机抽样：每个对象选入样本的概率相同，
    - 典型抽样：有意选取在具体特征上与更大总体相匹配的对象。假设我们把美国分为东部、中部和西部，从这三个部分各抽取2个州的数据，这种抽样方法就是典型抽样
    - 方便抽样：根据“方便程度”选择样本对象，如```df.head(5)```选取了前5行的数据作为样本

## 测量尺度

测量尺度决定了数据中蕴涵的信息量，并表明最适合的数据汇总和统计分析方法。

所谓的**测量尺度**，指的是依照一定的规则，用分类方法对事物的属性进行量化描述的过程。

例如，如果我们遇到开心的事情，我们可以用 1 - 10 来测量开心的程度，“用 1 - 10 来测量”这个动作，就是测量尺度

根据测量尺度的不同，我们将变量分为四种：

### 名义（定类）尺度

当一个变量的数据包含了用来识别个体属性的标记或名称时，测量尺度被称为名义尺度（nominal scale）。用无权重或无数值的符号，识别变量的不同类别

名义尺度指的是用来对变量进行分类的一种尺度，名义尺度不涉及量化数值或者事物的顺序，是四种度量类型中最简单的一种。由于不具有量化或者顺序，名义尺度无法进行计算。

名义尺度通常用于分类：不同变量的数值仅用于标识不同的类别，对这些数值进行加减没有任何实际意义。因此，变量的实际意义在于标识本身。

**名义尺度实例**

- 你的性别是？
    1. 男
    0. 女
- 政治倾向
    1. 无政治倾向
    2. 民主党
    3. 共和党
- 居住地
    1. 郊区
    2. 城区
    3. 乡镇

### 顺序（定序）尺度

如果数据具有名义数据的性质，并且数据的顺序或等级的意义明确，那么这种变量的测量尺度是顺序尺度（ordinal scale）。

顺序尺度通常用于描述变量的顺序，不包含变量之间的差异值。具体而言，顺序尺度只用于描述非数值的概念诸如频率、满意度、幸福指数、疼痛值等无法准确量化的指标。

顺序尺度包含两个维度的信息：描述性的分类（类似名义尺度）和变量固有的顺序。

顺序尺度比定类尺度能度量的信息量更大，因为其描述性分类不仅起到了分类的作用，也包含了变量的相对顺序。但顺序尺度无法对每一个指标之间的差距进行描述，因此，无法计算不同变量之间的差距。

**顺序尺度实例**

顺序尺度常用于测度**无法量化**，却**可以比较**的场景诸如工作状态、比赛排名、产品质量、满意度等。

1. 变量的顺序是极其重要的：非常不满意不如不满意，比较满意不如非常满意
2. 顺序尺度的所包含的信息量比名义尺度更多的原因就在于此：变量的顺序也包含了一定量的信息
3. 基于变量的顺序，进行数据分析
- 服务的满意度？
    1. 非常不满意
    2. 不满意
    3. 中立
    4. 比较满意
    5. 非常满意

其实，绝大多数专业的心理测量问卷，使用的尺度都是顺序尺度。

### 间隔（定距）尺度

如果数据具有顺序数据的所有性质，并且可以按某一固定度量单位表示数值间的间隔，则这种变量的测量尺度是间隔尺度（interval scale）。

间隔数据永远是数值型的。

### 比率（定比）尺度

如果数据具有间隔数据的所有性质，并且两个数值之比是有意义的，则这种变量的测量尺度是比率尺度（ratio scale）。

像距离、高度、重量和时间等变量都用比率尺度来度量。

比率尺度需要有一个零值，变量取零值时表示什么也不存在。

之所以把这两类变量放在一起，是因为他们之间的相似点足够多，而区别他们的方法也足够简单：

- 定距变量用有**权重**和**取值间距离**信息的符号来识别变量
- 定比变量和定距变量的定义相同，区别点就是：定比变量有**绝对零点**，即我们在使用变量的时候，假设变量是可以为 0 的
    - [绝对零点](https://www.zhihu.com/question/335726335)：客观存在的，不是人为设定的。如质量 = 0 意味着没有质量
    - 相对零点：不是客观存在的，而是人为规定的。如温度 = 0℃ 不意味着没有温度

下面我会用一张表格来展示四种变量的区别：

|     特征    | 定类 | 定序 | 定距 | 定比 |
|:----------:|:----:|:----:|:----:|:----:|
|  是否有频次  | Yes  | Yes  | Yes  | Yes  |
| 比较是否相等 |  Yes  |  Yes  | Yes  | Yes  |
| 变量是否有序 |  No  |**Yes**|Yes  | Yes  |
| 是否有中位数 |  No  |**Yes**|Yes  | Yes  |
|  是否有均值  |  No  |  No  | Yes  | Yes  |
| 比较大小 |  No  |  No  | Yes  | Yes  |
| 加减 |  No  |  No  | Yes  | Yes  |
| 乘除 |  No  |  No  |**No**| Yes  |
| 绝对零点 |  No  |  No  |**No**| Yes  |

## 分类（定类）型数据和数量（定量）型数据

根据变量是否可数，我们将变量分为**定量**和**定类**两种类型。
- 定量变量又称连续变量，用数字或者评分赋值，表示可计数的变量
- 定性变量又称分类变量，不可计数的变量
    - 二值变量：定性变量的一种，具有两个不同分类的定性变量，例如：男/女，是/否

归属于某一类别的数据称为分类型数据（categorical data），分类型数据既可以用名义尺度度量，也可以用顺序尺度度量。

用于表示大小或多少的数值称为数量型数据（quantitative data）。数值型数据既可以用间隔尺度度量，也可以用比率尺度度量。

## 截面数据和时间序列数据
为了便于统计分析，有必要对截面数据和时间序列数据进行区分。

截面数据（cross-sectional data）是在相同或近似相同的同一时点上搜集的数据。

时间序列数据（time series data）是在几个时期内搜集的数据。

## 描述统计和推断统计

### 描述统计
将数据以表格、图形或数值形式汇总的统计方法被称为描述统计（deseriptive statistics）

### 推断统计
在特定研究中，较大的个体组称为总体（population），较小的个体组称为样本（sample）。我们一般使用下列定义。

总体：总体是在一个特定研究中所有感兴趣的个体组成的集合。
样本：样本是总体的一个子集。

搜集总体全部数据的调查过程称为普查（census），搜集样本数据的调查过程称为抽样调查（sample survey）。

统计学的一个主要贡献就是**利用样本数据**对**总体特征**进行**估计**和**假设检验**，这一过程被称为统计推断（statistical inference）。

## 分析方法

### 逻辑分析方法
逻辑分析方法（analytics）是将数据转化为做出好决策见解的科学过程。

逻辑分析方法利用数据驱动或事实基础做出决策，在做决策时它通常被认为比其他方法更为客观。逻辑分析方法的工具能通过从数据中得出的见解辅助制定决策，提高我们更准确地预测计划的能力，帮助我们量化风险，以及通过分析产生更好的选择。

逻辑分析方法可以涉及从简单的报告到最先进的优化技术（寻找最佳行动过程的算法）等多种技术。现在逻辑分析方法一般被认为包含三大类别的技术，它们是描述性分析、预测性分析和规范性分析。

### 描述性分析

描述性分析（descriptive analytics）包含描述过去发生状况的分析技术集合。这类技术的例子是数据查询、报告、描述统计、数据可视化、数据仪表板和基本的“如果...则..."型电子表格模型。

### 预测性分析
预测性分析（predictive analytics）包含利用过去数据建立的模型来预测未来或评估一个变量对另一个变量的影响的分析技术。

例如，销售一种产品的过去数据可以用来建立一个预测未来销售的数学模型，这样的模型可以考虑诸如增长轨迹和基于过去的增长和季节模式的产品销售季节性等因素。零售商店的POS扫描数据可以被包装食品制造商利用，旨在估计与优惠券或销售事件相关联的单位货物的销售量。调查数据和过去的购买行为可以用来预测新产品的市场占有率。这些都是预测性分析的例子。

线性回归、时间序列分析和预测模型属于预测性分析，在本书之后的章节将讨论这些技术。模拟是为更好地了解风险的概率论和统计计算机模型的使用，也归属于预测性分析。

### 规范性分析
规范性分析有别于描述性分析和预测性分析。规范性分析的区别在于规范模型会产生一个最好的行动过程即规范模型的结果是最佳决策。

因此，规范性分析（prescriptive analytics）是**产生一个最佳行动过程的分析技术集合**。在一组约束条件下产生最大或最小目标的解决方案的优化模型属于规范性模型。

航空业营业管理的应用是规范模型的例子，航空业用过去的购买数据作为投入输入模型，得到最大限度地提高公司收入的所有航班的定价策略。

### 统计研究如何与逻辑分析方法有关？

大多数描述性和预测性分析技术来自概率论和统计学，其中包括描述统计、数据可视化、概率和概率分布、抽样以及预测建模（包括回归分析和时间序列预测）。

In [9]:
df = pd.read_csv(DATA/'PC.csv')
df

,平板电脑,价格（美元）,操作系统,显示器大小（英寸）,电池寿命（小时）,CPU厂商
0,Acer Iconia W510,599,Windows,10.10,8.50,英特尔
1,Amazon Kindle Fire HD,299,Android,8.90,9.00,德州仪器
2,Apple iPad 4,499,iOS,9.70,11.00,苹果
3,HP Envy X2,860,Windows,11.60,8.00,英特尔
4,Lenovo ThinkPad Tablet,668,Windows,10.10,10.50,英特尔
5,Microsoft Surface Pro,899,Windows,10.60,4.00,英特尔
6,Motorola Droid XYboard,530,Android,10.10,9.00,德州仪器
7,Samsung Atiy Smart PC,590,Windows,11.60,7.00,英特尔
8,Samsung Galaxy Tab,525,Android,10.10,10.00,英伟达
9,Sony Tablet S,360,Android,9.40,8.00,英伟达


- 这个数据集中有多少个体？
    - 10个个体
- 这个数据集中有多少变量？
    5个变量：
    - 价格（美元）
    - 操作系统
    - 显示器大小（英寸）
    - 电池寿命（小时）
    - CPU 厂商
- 哪些变量是分类变量，哪些变量是数量变量？
    - 分类变量：操作系统、CPU 厂商
    - 数量变量：价格（美元）、显示器大小（英寸）、电池寿命（小时）
- 每个变量使用哪种类型的测量尺度？
    - 价格、显示器大小和电池寿命是比例尺度
    - 操作系统和CPU厂商是名义尺度。

- 平板电脑的平均价格是多少？
    - 582.90美元

In [4]:
df['价格（美元）'].mean()

582.9

比较使用 Windows 操作系统和 Android 操作系统的平板电脑的平均价格？
- Windows 操作系统：723.20美元
- Android 操作系统：428.5美元

In [10]:
df.query("操作系统 != 'iOS'").groupby('操作系统')['价格（美元）'].mean()
#df.query("操作系统 in ('Windows','Android')").groupby('操作系统').mean()

操作系统
Android   428.50
Windows   723.20
Name: 价格（美元）, dtype: float64